In [3]:
from jupyterthemes import jtplot

jtplot.style(theme = 'oceans16', context='paper')

# 1. Título

# 2. Descripción del tema

# 3. Justificación

# 4. Descripción del Corpus

# 5. Plan de análisis

# 6. Análisis de texto

# 7. Conclusiones